In [4]:
import os
import cv2
import glob 
import numpy as np
import pandas as pd
import mediapipe as mp
import face_recognition
import threading

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils


In [2]:
images_dir_path = 'faces'
data_path = "face_data.npz"
df_lock = threading.Lock()

try:
    loaded_data = np.load(data_path)
    known_face_names = list(loaded_data['names'])
    known_face_encodings = list(loaded_data['encodings'])
except:
    images_path = glob.glob(f'{images_dir_path}/*')
    known_face_names = []
    known_face_encodings = []
    for img_path in images_path:
        face_name = os.path.splitext(os.path.basename(img_path))[0]
        
        face_image = face_recognition.load_image_file(img_path)
        face_encoding = face_recognition.face_encodings(face_image)[0]
        
        known_face_names.append(face_name)
        known_face_encodings.append(np.array(face_encoding))
    np.savez_compressed(data_path, names=np.array(known_face_names), encodings=np.array(known_face_encodings))

print(known_face_names)
# print(known_face_encodings)

['elon', 'federico']


In [3]:
def identification(frame, face_location):
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_location)

    face_names = []
    for face_encoding in face_encodings:

        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        face_distances = face_recognition.face_distance(
                        known_face_encodings, face_encoding
                    )
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        face_names.append(name)
        
    return face_names


def get_centered_face(image):
    with mp_face_detection.FaceDetection(
        min_detection_confidence=0.5
    ) as face_detection:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)
        if results.detections:
            h, w, _ = image.shape
            center_x = w / 2
            min_distance = float("inf")
            centered_face = None
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                x_center = bboxC.xmin + bboxC.width / 2
                distance = abs(x_center - center_x)
                if distance < min_distance:
                    min_distance = distance
                    centered_face = detection
            if centered_face:
                bboxC = centered_face.location_data.relative_bounding_box
                x1, y1, x2, y2 = (
                    int(bboxC.xmin * w),
                    int(bboxC.ymin * h),
                    int((bboxC.xmin + bboxC.width) * w),
                    int((bboxC.ymin + bboxC.height) * h),
                )
                return [(y1, x2, y2, x1)]
    return None


def draw_rect_on_face(image, face_location, face_names):
    if face_location:
        for (top, right, bottom, left), name in zip(face_location, face_names):
            # top, right, bottom, left = face_location[0]
            cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(
                image,
                name,
                (left, bottom + 25),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),
                2,
            )
            cv2.imshow("Face Recognition", image)

def add_face(frame):
    face_name = input('entre face name')
    face_encoding = face_recognition.face_encodings(frame)[0]
    known_face_names.append(face_name)
    known_face_encodings.append(np.array(face_encoding))
    np.savez_compressed(data_path, names=np.array(known_face_names), encodings=np.array(known_face_encodings))

In [5]:
# Set up the webcam
cap = cv2.VideoCapture(0)
face_names = []
while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    face_location = get_centered_face(frame)
    if not face_location:
        print("no central face detected")
        break

    key = cv2.waitKey(1)
    if key == ord("c"):
        print("<< -C- pressed >>")
        face_names = identification(frame, face_location)
        key = None
        
    if len(face_names)== 1 and face_names[0] == "Unknown":
        face_name = input('entre face name')
        face_encoding = face_recognition.face_encodings(frame)[0]
        known_face_names.append(face_name)
        known_face_encodings.append(np.array(face_encoding))
        np.savez_compressed(data_path, names=np.array(known_face_names), encodings=np.array(known_face_encodings))
        face_names[0] = face_name
    else:
        draw_rect_on_face(frame, face_location, face_names)
        
    # Display the image and wait for the "c" key to be pressed to trigger identification again
    if not face_names or key != ord("c"):
        cv2.imshow("Face Recognition", frame)

    key = cv2.waitKey(1)
    if key == ord("q"):
        cv2.destroyAllWindows()
        key = None
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


<< -C- pressed >>


In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import mediapipe as mp
import face_recognition
import threading

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils


images_dir_path = "faces"
data_path = "face_data.npz"

try:
    loaded_data = np.load(data_path)
    known_face_names = list(loaded_data["names"])
    known_face_encodings = list(loaded_data["encodings"])
except:
    images_path = glob.glob(f"{images_dir_path}/*")
    known_face_names = []
    known_face_encodings = []
    for img_path in images_path:
        face_name = os.path.splitext(os.path.basename(img_path))[0]

        face_image = face_recognition.load_image_file(img_path)
        face_encoding = face_recognition.face_encodings(face_image)[0]

        known_face_names.append(face_name)
        known_face_encodings.append(np.array(face_encoding))
    np.savez_compressed(
        data_path,
        names=np.array(known_face_names),
        encodings=np.array(known_face_encodings),
    )


face_locations = []
face_encodings = []
face_names = []
process_current_frame = True
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_current_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(
            rgb_small_frame, face_locations
        )

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(
                known_face_encodings, face_encoding
            )

            # Calculate the shortest distance to face
            face_distances = face_recognition.face_distance(
                known_face_encodings, face_encoding
            )

            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            else:
                name = "Unknown"
            
            face_names.append(name)

    process_current_frame = not process_current_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Create the frame with the name
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(
            frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED
        )
        cv2.putText(
            frame,
            name,
            (left + 6, bottom - 6),
            cv2.FONT_HERSHEY_DUPLEX,
            0.8,
            (255, 255, 255),
            1,
        )

    key = cv2.waitKey(1)
    if key == ord("c"):
        print("<< -C- pressed >>")
        if face_names and face_names[0] == "Unknown":
            face_name = input("entre face name")
            face_encoding = face_recognition.face_encodings(frame)[0]
            known_face_names.append(face_name)
            known_face_encodings.append(np.array(face_encoding))
            np.savez_compressed(
                data_path,
                names=np.array(known_face_names),
                encodings=np.array(known_face_encodings),
            )
            face_names[0] = face_name

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) == ord("q"):
        cv2.destroyAllWindows()
        break

# Release handle to the webcam
video_capture.release()
